In [ ]:
import os, warnings
from functools import partial

import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import geometry

# utils
from joblib import Parallel, delayed
from pandas import IndexSlice as idx
from IPython.display import display

# viz
import folium

# local imports
from spatial_interpolation import data, utils
from spatial_interpolation.visualization import map_viz
from spatial_interpolation.utils import tqdm_joblib

from experiments.noaa.machine_learning.feature_extraction import FeatureExtractionExperiment
from joblib import Parallel, delayed

import dotenv

# configs
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
dotenv.load_dotenv()

### Load the data available:

In [ ]:
buoy_df, buoy_gdf = data.load_buoy_data(period=("2011-01-01", "2022-01-01"))
world_countries = data.load_world_borders()
world_countries = world_countries[world_countries.overlaps(geometry.box(*buoy_gdf.total_bounds.tolist()))]
# Remove the "buoys" that appear to be in land
land_buoys = buoy_gdf.geometry.apply(lambda x: any(world_countries.intersects(x,align=False)))
buoy_gdf = buoy_gdf[~land_buoys]
buoy_df = buoy_df[buoy_df.index.get_level_values("buoy_id").isin(buoy_gdf.index.get_level_values("buoy_id").unique())]

### Run Feature Extraction Experiment(s)

In [ ]:
import subprocess
set_conf = "set1"
for y in range(2011,2022):
    experiment_name = FeatureExtractionExperiment.__name__
    p = subprocess.Popen(["python", "-m", "experiments", experiment_name , set_conf, f"--year={y}"])
    print(f"Started experiment {experiment_name} with config {set_conf} and year {y} on process {p.pid}")
    p.communicate()

In [ ]:
def run_make_features_at_year(conf, year, **kwargs):
    experiment = FeatureExtractionExperiment(conf, year=year, **kwargs)
    experiment.set_data((buoy_df, buoy_gdf))
    _ = experiment.run()
    return _

In [ ]:
from IPython.utils import io

set_conf = "set1"
# with io.capture_output() as captured:
for y in range(2011,2022):
    print(f"Extracting features on {set_conf} for year {y}")
    run_make_features_at_year(set_conf, y, verbose=True)#, num_jobs=02)

### Analysis

In [ ]:
config = FeatureExtractionExperiment("set1").get_config()
train_df = pd.concat(
    [pd.read_parquet(f"{config.output.train_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()
test_df = pd.concat(
    [pd.read_parquet(f"{config.output.eval_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()

In [ ]:
locations_within_area = buoy_gdf.loc[buoy_gdf.within(config.area)].index.get_level_values("buoy_id").unique()
print("Training set:")
print("Shape:", train_df.shape)
print("All within area?",train_df.index.get_level_values("location_id").isin(locations_within_area).all())
print("Any eval buoys in train?",train_df.index.get_level_values("location_id").isin(config.locations_full).sum())
print("Any eval index in train?",train_df.index.isin(config.split_strategy.params.eval).sum())

print("Eval set:")
print("Shape:", test_df.shape)
print("All within area?",test_df.index.get_level_values("location_id").isin(locations_within_area).all())
print("Any eval buoys?",test_df.index.get_level_values("location_id").isin(config.locations_full).sum())
print("Any eval index?",test_df.index.isin(config.split_strategy.params.eval).sum())

### Viz:

In [ ]:
# pd.options.plotting.backend = "plotly"
fig = (
    buoy_df
    .loc[idx["42019","1995-01-01":],["wave_height"]]
    .loc["42019"]
    .resample("5D").mean()
    .plot(title=\
        f"Historical of Wave Height for Buoy {buoy_gdf.loc[2011,'42019'].buoy_name}",
        # labels=dict(value="Wave Height (meters)",variable=""),
    )
)
fig

In [ ]:
set_1_buoys = ["41036","42040","42065","41041"]
entire_area = geometry.box(*(-98.251934,12.282308,-45,35.55)) 
area_1 = geometry.box(*(-97.3,18.930645,-81.2,30.366655))
area_2 = geometry.box(*(-82.836914,25.799891,-68.115234,35.55))
box1_gdf = gpd.GeoDataFrame({"geometry":[area_1.union(area_2)]},crs="epsg:4326")
outer_box_gdf = gpd.GeoDataFrame({"geometry":[entire_area]},crs="epsg:4326")

available_buoys = buoy_df.index.get_level_values("buoy_id").unique()
available_buoys_gdf = buoy_gdf.loc[buoy_gdf.index.get_level_values("buoy_id").isin(available_buoys)].reset_index()
uniques = buoy_gdf.loc[idx[:,set_1_buoys],:].reset_index().drop_duplicates(subset="buoy_id",keep="last").set_index("buoy_id")
mp = map_viz.make_map_of_buoys(
    location = available_buoys_gdf.unary_union.centroid.coords[0][::-1],
    zoom_start = 4,
    buoy_locations_df=available_buoys_gdf, 
    marker=partial(folium.CircleMarker, radius=1, weight=5,color="black"),
)
map_viz.add_geodf_to_map(uniques.loc[set_1_buoys],map=mp, color="red", radius=1, weight=5, layer_name="buoys from eval set 1")
area_1_gdf = gpd.GeoDataFrame({"geometry":[area_1]},crs="epsg:4326")
map_viz.add_geodf_to_map(utils.flip_coords(area_1_gdf),map=mp, layer_name="box2", color="yellow", alpha=0.5)
map_viz.add_geodf_to_map(utils.flip_coords(box1_gdf),map=mp, layer_name="box1",color="red")
map_viz.add_geodf_to_map(utils.flip_coords(outer_box_gdf),map=mp, layer_name="outer box", color="blue")
folium.LayerControl().add_to(mp)
mp

In [ ]:
# locations with the most data points
most_points = buoy_df[["wave_height","wind_direction"]].groupby(
    "buoy_id"
).count().sort_values(
    by=["wave_height","wind_direction","buoy_id"],
    ascending=False)
print("Locations with the most data points:")
display(most_points.head(10))
# times with the most data points
most_points = buoy_df[["wave_height","wind_direction"]].groupby(
    "time"
).count().sort_values(
    by=["wave_height","wind_direction","time"],
    ascending=False)
print("Times with the most data points:")
display(most_points.head(10))

---